# Nemo for speaker recognition 
find nemo here - https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/index.html

tryin nemo by nvedia to predict the animal species nemo has pretrained models and is easy to transfer learn things. in this notebook I am trying to use nemo for detect the animal species mostly nemo has good performance In asr.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as mpl_patches
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 8),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large',
         'text.color' : "green",
          
         }
pylab.rcParams.update(params)

import torch
import torchaudio
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torchvision.models import resnet34
import torch
import torch.nn as nn
import torch.optim as optim
import librosa
from tqdm.notebook import tqdm

from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
import IPython.display as ipd
from IPython.display import display, HTML

In [ ]:
train_path = Path("../input/rainforest-wav-format/train_wav_format/")
test_path = Path("../input/rainforest-wav-format/test_wav_format/")
train_recs = list(train_path.glob("*.wav"))
test_recs = list(test_path.glob("*.wav"))

In [ ]:
train = pd.read_csv('/kaggle/input/rfcx-species-audio-detection/train_tp.csv')


In [ ]:
train

In [ ]:
# lets make a manifest for nemo training
# {"audio_filepath": "/content/data/an4/wav/an4_clstk/mjda/cen6-mjda-b.wav", "duration": 2.0, "label": "mjda"}

In [ ]:
import librosa
import IPython.display as ipd

# Load and listen to a random audio file
example_file = '../input/rainforest-wav-format/train_wav_format/0099c367b.wav'
audio, sample_rate = librosa.load(example_file)
#lets hear the audio
ipd.Audio(example_file, rate=sample_rate)

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
X = train.drop(['species_id'] , axis = 1)
y = train['species_id'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 , stratify = y)




In [ ]:
y_train

In [ ]:
y_test

In [ ]:
train_manifest_data = X_train.copy()
train_manifest_data['species_id'] = y_train

test_manifest_data = X_test.copy()
test_manifest_data['species_id'] = y_test


In [ ]:
train_manifest_data[['recording_id' , 'species_id']]

In [ ]:
'''train manifest lets first pass the whole audio recording 
which doesnt seem right but we can improve once we setup the whole pipeline'''
from scipy.io import wavfile
import json
manifest_path = 'train_manifest.json'
os.mkdir('train_trim') 
os.mkdir('test_trim') 

with open(manifest_path, 'w') as fout:
    for i,r,s,tmax,tmin in train_manifest_data[['recording_id' , 'species_id' ,'t_max','t_min']].itertuples():
        audio_path = str(train_path)+'/'+r+'.wav'
        
        sampleRate, waveData = wavfile.read( audio_path )
        startSample = int( tmin * sampleRate )
        endSample = int( tmax * sampleRate )
        wavfile.write( 'train_trim/'+r+"_"+str(i)+'.wav' , sampleRate, waveData[startSample:endSample])
        # converting to 16khz from 48khz
        y, sr = torchaudio.load('train_trim/'+r+"_"+str(i)+'.wav')
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(sr, 16000)
            y_resampled = resampler(y)
        torchaudio.save('train_trim/'+r+"_"+str(i)+'.wav' ,y_resampled,16000)

        duration = librosa.core.get_duration(filename='train_trim/'+r+"_"+str(i)+'.wav' )
        metadata = {
                    "audio_filepath": 'train_trim/'+r+"_"+str(i)+'.wav',
                    "duration": duration,
                    "label": s
                }
        json.dump(metadata, fout)
        fout.write('\n')
    
    
    


In [ ]:
# test manifest
manifest_path = 'test_manifest.json'

with open(manifest_path, 'w') as fout:
    for i,r,s,tmax,tmin in test_manifest_data[['recording_id' , 'species_id' ,'t_max','t_min']].itertuples():
        audio_path = str(train_path)+'/'+r+'.wav'
        
        sampleRate, waveData = wavfile.read( audio_path )
        startSample = int( tmin * sampleRate )
        endSample = int( tmax * sampleRate )
        wavfile.write( 'test_trim/'+r+"_"+str(i)+'.wav' , sampleRate, waveData[startSample:endSample])
        # converting to 16khz from 48khz
        y, sr = torchaudio.load('test_trim/'+r+"_"+str(i)+'.wav')
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(sr, 16000)
            y_resampled = resampler(y)
        torchaudio.save('test_trim/'+r+"_"+str(i)+'.wav' ,y_resampled,16000)

        duration = librosa.core.get_duration(filename='test_trim/'+r+"_"+str(i)+'.wav' )
        metadata = {
                    "audio_filepath": 'test_trim/'+r+"_"+str(i)+'.wav',
                    "duration": duration,
                    "label": s
                }
        json.dump(metadata, fout)
        fout.write('\n')
    

In [ ]:
# setup nemo
BRANCH = 'r1.0.0b3'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]
NEMO_ROOT = os.getcwd()

In [ ]:
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr
from omegaconf import OmegaConf

In [ ]:
# This line will print the entire config of sample SpeakerNet model

!mkdir conf 
!wget -P conf https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/speaker_recognition/conf/SpeakerNet_recognition_3x2x512.yaml
MODEL_CONFIG = os.path.join(NEMO_ROOT,'conf/SpeakerNet_recognition_3x2x512.yaml')
config = OmegaConf.load(MODEL_CONFIG)
print(OmegaConf.to_yaml(config))

In [ ]:
# setup manifest
train_manifest = 'train_manifest.json'
validation_manifest = 'test_manifest.json'
test_manifest = 'test_manifest.json'

In [ ]:
config.model.train_ds.manifest_filepath = train_manifest
config.model.validation_ds.manifest_filepath = validation_manifest
config.model.test_ds.manifest_filepath = validation_manifest

In [ ]:
config.model.decoder.num_classes = 24

In [ ]:
import torch
import pytorch_lightning as pl

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
torch.cuda.is_available()

In [ ]:
# Let us modify some trainer configs for this demo
# Checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda


config.trainer.max_epochs = 15

# Remove distributed training flags
config.trainer.accelerator = None

In [ ]:
trainer = pl.Trainer(**config.trainer)

In [ ]:
from nemo.utils.exp_manager import exp_manager
log_dir = exp_manager(trainer, config.get("exp_manager", None))
# The log_dir provides a path to the current logging directory for easy access
print(log_dir)

In [ ]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel(cfg=config.model, trainer=trainer)

In [ ]:
trainer.fit(speaker_model)

validation loss is all over the place lets try to fix it

In [ ]:
# lets optimize it
trainer.test(speaker_model, ckpt_path=None)

In [ ]:
# Let us list all the checkpoints we have
from glob import glob
checkpoint_dir = os.path.join(log_dir, 'checkpoints')
checkpoint_paths = list(glob(os.path.join(checkpoint_dir, "*.ckpt")))
checkpoint_paths

In [ ]:
final_checkpoint = list(filter(lambda x: "-last.ckpt" in x, checkpoint_paths))[0]
print(final_checkpoint)

In [ ]:
# Restoring from a PyTorch Lightning checkpoint
restored_model = nemo_asr.models.EncDecSpeakerLabelModel.load_from_checkpoint(final_checkpoint)

In [ ]:
# restored_model.forward(input_signal = 'test_trim/11bafff5d_74.wav',input_signal_length = 2.352 )

In [ ]:
# restored_model.forward(input_signal =  y ,input_signal_length = (2.352) )
# (input_signal = 'test_trim/11bafff5d_74.wav',input_signal_length= 2.352)
    

In [ ]:
# lets get the nemo file
restored_model.save_to(os.path.join(log_dir, '..',"SpeakerNet.nemo"))

In [ ]:
verification_model = nemo_asr.models.ExtractSpeakerEmbeddingsModel.restore_from(os.path.join(log_dir, '..', 'SpeakerNet.nemo'))

In [ ]:
# Now, we need to pass the necessary manifest_filepath and params to set up the data loader for extracting embeddings
# lets check first five cases for demo
!head -5 {validation_manifest} > embeddings_manifest.json


In [ ]:
config.model.train_ds

In [ ]:
test_config = OmegaConf.create(dict(
    manifest_filepath = os.path.join(NEMO_ROOT,'embeddings_manifest.json'),
    sample_rate = 16000,
    labels = None,
    batch_size = 1,
    shuffle = False,
    time_length = 8,
    embedding_dir='./'
))
print(OmegaConf.to_yaml(test_config))
verification_model.setup_test_data(test_config)

In [ ]:
trainer = pl.Trainer(gpus=cuda,accelerator=None)
trainer.test(verification_model)

In [ ]:
ls embeddings/
